<h1 style="color:green">NBU-powered Scratchpad</h1>
<p style="color:green">This notebook is used for code smithening purposes</p>
<p style="color:green">It includes notebook utilities support, which means it can easily invoke functionality from the Conway-based project it belongs to</p>
<p>NB: this notebook requires to be launched in an environment that includes conway and conway.ops</p>

In [1]:
import os                                                               as _os
import sys
sys.path.append(_os.path.dirname(_os.path.dirname(_os.getcwd())) + "/src") # So we can import scratch_nb_apps
from scratch_nb_apps.scratch_nb_utils import Scratch_NB_Utils
NBU                       = Scratch_NB_Utils()
DFU                       = NBU.DFU
T0                        = NBU.time.perf_counter()

SCRATCH installation:                scratch_fork    
Jupyter using repo[branch]:      scratch.ops[afin-dev]    
Installation path:               /home/alex/consultant1@CCL/dev/scratch_fork    
Application:                     <class 'scratch_nb_apps.scratch_nb_application.Scratch_NB_Application'>    


<h1 style="color:purple">Graph of Awaitables</h1>
<p> We pilot whether we can maintain a graph of awaitables, including:</p>
<li> Multiple dependencies: awaitables B, C both awaiting on a common awaitable A

In [2]:
import asyncio

In [53]:
async def A(prefix="") -> str:
    name = f"{prefix}A"
    print(f"{name} starts ...", flush=True)
    await asyncio.sleep(1)
    print(f"... {name} ends", flush=True)
    return f"{name}"

In [54]:
async def B(prefix="") -> str:
    name = f"{prefix}B"
    print(f"{name} starts ...", flush=True)
    x = await A(f"{name}->")
    print(f"... {name} ends", flush=True)
    return f"{name}"

In [55]:
async def C(prefix="") -> str:
    name = f"{prefix}C"
    print(f"{name} starts ...", flush=True)
    x = await B(f"{name}->")
    print(f"... {name} ends", flush=True)
    return f"{name}"

In [152]:
class UsheringTo():

    def __init__(self, result_l):
        self.result_l = result_l
        #self._to_do = []

    def __iadd__(self, coro):
        if not asyncio.iscoroutine(coro):
            raise ValueError(f"Expected a coroutine, but instead got a {type(coro)}")
        self.to_do.append(coro)
        return self

    async def __aenter__(self):
        self.to_do = []
        return self

    async def __aexit__(self, exc_type, exc_value, traceback):
        to_do_iter                                      = asyncio.as_completed(self.to_do)
        for coro in to_do_iter:
            coro_result                                 = await coro
            self.result_l.append(coro_result)
        return

In [154]:
result_l = []
work_dict = {"a": A, "b": B, "c": C}

async with UsheringTo(result_l) as usher:
    for key in work_dict.keys():
        usher += work_dict[key]()
        
result_l
    
    

B starts ...
B->A starts ...
C starts ...
C->B starts ...
C->B->A starts ...
A starts ...
... B->A ends
... B ends
... C->B->A ends
... C->B ends
... C ends
... A ends


['B', 'C', 'A']

In [ ]:
async def supervisor():
    result_l                                        =  []
    print("Scheduling A ...", flush=True)
    a = A()
    print("... A scheduled", flush=True)
    print("Scheduling B ...", flush=True)
    b = B()
    print("... B scheduled", flush=True)
    print("Scheduling C ...", flush=True)
    c = C()
    print("... C scheduled", flush=True)
    to_do = [a, b, c]
    to_do_iter                                      = asyncio.as_completed(to_do)
    for coro in to_do_iter:
        coro_result                                 = await coro
        result_l.append(coro_result)
    return result_l

In [110]:
#result_l = asyncio.run(supervisor()) # Don't do this in Jupyter since Jupyter is already running an event loog
result_l = await supervisor()
result_l

Scheduling A ...
... A scheduled
Scheduling B ...
... B scheduled
Scheduling C ...
... C scheduled
C starts ...
C->B starts ...
C->B->A starts ...
A starts ...
B starts ...
B->A starts ...
... C->B->A ends
... C->B ends
... C ends
... A ends
... B->A ends
... B ends


['C', 'A', 'B']